In [1]:
#得到原始IDList, fileList (遍历scop-2.07-40文件夹)
#结合errorList 做差集得到 IDList
import os
from pathlib import Path

dir_path = "/home/wngys/scop/scope-2.07-40"

IDList = []
fileList = []

for child_path in os.listdir(dir_path):
    child_path = os.path.join(dir_path, child_path)
    files = sorted([os.path.join(child_path, f_path) for f_path in os.listdir(child_path) if f_path.endswith(".ent")])
    for f_path in os.listdir(child_path):
        if f_path.endswith(".ent"):
            IDList.append(f_path.split('.')[0])
            fileList.append(os.path.join(child_path, f_path))
            
IDset = set(IDList)

In [2]:
import numpy as np

np.save("/home/wngys/lab/DeepFold/protein_infor/IDArray.npy", IDList)
np.save("/home/wngys/lab/DeepFold/protein_infor/entPathArray.npy", fileList)

In [ ]:
# 根据蛋白质ID得到具体文件路径
idx = IDList.index("d3m2pa1")
print(fileList[idx])

In [ ]:
#分析align40文件 第一列集合与第二列集合 交集与并集

file_name = "/home/wngys/scop/TMalign/tmalign40.txt"
# file_name = "1.txt"
set1 = set()
set2 = set()
with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]
        # id3 = lines.split('\t')[2]
        set1.add(id1)
        set2.add(id2)

print(len(set1))
print(len(set2))

set3 = set.union(set1, set2)
set4 = set.intersection(set1, set2)

print(len(set3))
print(len(set4))

In [ ]:
# 筛选得到 valid_pair

file_name = "/home/wngys/scop/TMalign/tmalign40.txt"
out_file = "valid_pair.txt"
f_out = open(out_file, "w")
# file_name = "1.txt"

with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]

        if id1 in IDset and id2 in IDset:
            f_out.write(lines)
f_out.close()

In [ ]:
# 得到valid_set集合(align文件第一列蛋白质集合)
file_name = "valid_pair.txt"
valid_set = set()
with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]
        valid_set.add(id1)

print(len(valid_set))

14256


In [ ]:
# valid_set 由valid_pair的第一列或第二列集合操作得到
print(len(IDset - valid_set))

In [ ]:
print(len((IDset & set1) - set2))
print(len((IDset & set2) - set1))
print(len(IDset & set1 & set2))

In [ ]:
print(len(set.intersection(IDset, set4)))
print(len(set.union(IDset, set4)))

In [ ]:
# 根据valid_pair文件，将align文件左侧一列 按蛋白质类别 分成许多txt
file_name = "valid_pair.txt"
# file_name = "pair_/d1b77a2.txt"
dir_name = "Pair/"
dic = {}
with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]
        score = lines.split('\t')[2].split('\n')[0]
        if id1 not in dic:
            dic[id1] = []
        dic[id1].append((id2, score))

for id in dic.keys():
    f_name = dir_name + id + ".txt"
    f_out = open(f_name, "w")
    for ele in dic[id]:
        f_out.write(ele[0])
        f_out.write('\t')
        f_out.write(ele[1])
        f_out.write("\n")
    f_out.close()

In [ ]:
# 验证一下Pair文件夹 文件数目是否等于align文件左侧蛋白质集合 元素数目
dir = "Pair"

files_n = os.listdir(dir)

print(len(files_n))

In [ ]:
# 由Pair文件 将第二列TM-score转换为 bool 表示 positive or negtive
dir_pair = "Pair"
out_dir = "Pair_bool/"
p = 0.9
i = 0
for file_n in os.listdir(dir_pair):
    # if i >= 5:
    #     break
    file_name = os.path.join(dir_pair, file_n)
    Max_score = -1
    with open(file_name, "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            # id2 = lines.split('\t')[0]
            score = lines.split('\t')[1].split("\n")[0]
            score = float(score)
            Max_score = max(score, Max_score)

    threshold = Max_score *p
    # print(file_n)
    # print(f"Max_score:{Max_score}")
    f_w = open(out_dir + file_n, "w")

    with open(file_name, "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            id2 = lines.split('\t')[0]
            score = lines.split('\t')[1].split("\n")[0]
            score = float(score)
            f_w.write(id2)
            f_w.write("\t")

            if score >= threshold:
                # print(id2, score)
                f_w.write(str(1))
                f_w.write("\n")
            else:
                f_w.write(str(0))
                f_w.write("\n")
    f_w.close()

    # i += 1
    # print("------------------------------------------------")